In [1]:
!python --version

Python 3.10.9


In [2]:
!pip install torch
!pip install torchvision
!pip install ftfy regex tqdm

In [3]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\ching\appdata\local\temp\pip-req-build-n4_ko31l
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Ching\AppData\Local\Temp\pip-req-build-n4_ko31l'


# Get food data from csv

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
food = pd.read_csv("out.csv")
food.drop_duplicates(subset='sample name', keep='first', inplace=True)
food = food.reset_index(drop=True)
food_name = food['sample name']
food_feature = food['eng text']
food_feature.head(10)

0    coix seed has a light brown color and a slight...
1    rolled oats  is a type of cereal made from cru...
2    coix seed powder 's color is light beige with ...
3    wheat tea  appears as a light-brown colored li...
4    millet  is a type of grain that is typically y...
5    glutinous rice is a type of glutinous rice tha...
6    wheat  (Xiaomai) are small steamed dumplings w...
7    wheat germ is light brown in color and has a s...
8    gluten-free flour or de-glutenized flour  has ...
9    low gluten flour  has a pale white color and a...
Name: eng text, dtype: object

In [5]:
food

,樣品名稱,sample name,eng text
0,大麥仁,coix seed,coix seed has a light brown color and a slight...
1,大麥片,rolled oats,rolled oats is a type of cereal made from cru...
2,大麥仁粉,coix seed powder,coix seed powder 's color is light beige with ...
3,麥茶,wheat tea,wheat tea appears as a light-brown colored li...
4,小米,millet,millet is a type of grain that is typically y...
...,...,...,...
2162,NaN,turnip cake,turnip cake has a pale yellow color and a smoo...
2163,NaN,twist roll,twist rolls are golden brown in color with a c...
2164,NaN,wheel pie,the wheel pie has a golden-brown color on its ...
2165,NaN,xiaolongbao,xiaolongbao is a type of steamed dumpling that...


In [6]:
food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2167 entries, 0 to 2166
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   樣品名稱         2067 non-null   object
 1   sample name  2167 non-null   object
 2   eng text     2167 non-null   object
dtypes: object(3)
memory usage: 50.9+ KB


In [7]:
# max_length = 250
# for i in range(0,len(food_feature)):
#     if len(food_feature[i]) > max_length:
#         food_feature[i] = food_feature[i][:max_length]

In [8]:
text_snippets = []
Text_map = {}
idx = 0
checking_arr = []
print(food.shape[0])
for i in range(0,food.shape[0]):
  #str(TN_eng_name[i]).strip(',')
  if(food_name[i] in checking_arr):
    continue
  else:
    checking_arr.append(str(food_name[i]))
    Text_map[idx] = str(food_name[i])
    text_snippets.append(food_feature[i])
    idx += 1
print(idx)
# print(Text_map.values())
# print(text_snippets)

2167
2167


# CLIP model

In [85]:
import torch
import clip

In [86]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# load model and img preprocessing
model, preprocess = clip.load("ViT-B/32", device=device,jit = False)

### Get image from google

In [87]:
# !pip install google_images_download
# !pip install python-magic-bin
# !pip install simple_image_download
# !pip install --upgrade simple_image_download

In [88]:
# from google_images_download import google_images_download

# response = google_images_download.googleimagesdownload()
# arguments = {"keywords": "dog",
#              "limit": 10,
#              "print_urls": True}
# paths = response.download(arguments)
# print(paths)

In [89]:
from PIL import Image
import requests

# create df to save result
num_rows = 101
out = pd.DataFrame(index=range(num_rows))
out['output'] = np.nan

In [ ]:
# 文本更新再重跑就好
# 預處理文本
text = clip.tokenize(text_snippets, context_length=150).to(device)
print(text.shape)

with torch.no_grad():
  text_features = model.encode_text(text)
print(text_features.shape)

In [ ]:
# 加載圖片
for i in range(1,102):
    f = 'pic/{}.jpg'.format(i)
    image = Image.open(f)
    print("Image to be processed")
    # display(image)

    # 預處理圖像
    image = preprocess(image).unsqueeze(0).to(device)
    print("\n\nTensor shape:")
    print(image.shape)
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        print(image_features.shape)

    with torch.no_grad():
        logits_per_image, logits_per_text = model(image,text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    
    import numpy as np
    import heapq

    #np.argmax(probs)
    max3 = map(list(probs[0]).index, heapq.nlargest(5, probs[0]))
    max3 = list(max3)
    print(max3)
    # print('1. '+Text_map[max3[0]])
    # print('2. '+Text_map[max3[1]])
    # print('3. '+Text_map[max3[2]])
    # print('4. '+Text_map[max3[3]])
    # print('5. '+Text_map[max3[4]])

    out['output'][i-1] = '1.'+ Text_map[max3[0]] +' 2.'+ Text_map[max3[1]] +' 3.'+ Text_map[max3[2]] +' 4.'+ Text_map[max3[3]] +' 5.'+ Text_map[max3[4]] 
    #Text_map[np.argmax(probs)]

out.to_csv('output.csv', header='output', index=False, encoding='utf_8_sig')

In [178]:
# index
print(max3)

[944, 2164, 1005, 2187, 1019]
